In [2]:
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d vikramtiwari/pix2pix-dataset

from zipfile import ZipFile
dataset_path = './pix2pix-dataset.zip'

with ZipFile(dataset_path, 'r') as z:
    z.extractall(path='./dataset')
    print("Done")

Saving kaggle.json to kaggle.json
100% 2.40G/2.40G [02:03<00:00, 22.1MB/s]
100% 2.40G/2.40G [02:03<00:00, 20.9MB/s]
Done


In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

import albumentations as A
from albumentations.pytorch import ToTensorV2

import os
import shutil
import numpy as np
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt

In [2]:
class convBlockDisc(nn.Module):
    def __init__(self, in_channels, out_channels, stride):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=stride, padding=1, bias=False, padding_mode="reflect"),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2),
        )
    
    def forward(self, x):
        return self.conv(x)

In [3]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3, channels=[64, 128, 256, 512]):
        super().__init__()
        self.initial = nn.Sequential(
            nn.Conv2d(in_channels*2, channels[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        layers = []
        in_channels = channels[0]
        for channel in channels[1:]:
            layer = convBlockDisc(in_channels, channel, stride=2 if channel != channels[-1] else 1)
            layers.append(layer)
            in_channels = channel

        self.layers = nn.Sequential(*layers)
        self.final = nn.Conv2d(channels[-1], 1, kernel_size=4, stride=1, padding=1, padding_mode="reflect")
    
    def forward(self, x, y):
        x = torch.cat((x, y), dim=1)
        x = self.initial(x)
        x = self.layers(x)
        x = self.final(x)
        return x

In [4]:
x = torch.randn((8, 3, 256, 256))
y = torch.randn((8, 3, 256, 256))
disc = Discriminator()
disc(x, y).shape

torch.Size([8, 1, 30, 30])

In [5]:
class convBlockGen(nn.Module):
    def __init__(self, in_channels, out_channels, down=True, act="relu", use_dropout=False):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False, padding_mode="reflect") if down
            else nn.ConvTranspose2d(in_channels, out_channels, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU() if act == "relu" else nn.LeakyReLU(0.2),
        )
        self.use_dropout = use_dropout
        self.dropout = nn.Dropout(0.5)
    
    def forward(self, x):
        x = self.conv(x)
        x = self.dropout(x) if self.use_dropout else x
        return x

In [6]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=3, channels=[64, 128, 256, 512, 512, 512, 512]):
        super().__init__()
        self.initial_down = nn.Sequential(
            nn.Conv2d(in_channels, channels[0], kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.LeakyReLU(0.2),
        )

        self.downs = nn.ModuleList()
        in_channels = channels[0]
        for channel in channels[1:]:
            down = convBlockGen(in_channels, channel, down=True, act="leaky", use_dropout=False)
            self.downs.append(down)
            in_channels=channel

        self.bottleNeck = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=4, stride=2, padding=1, padding_mode="reflect"),
            nn.ReLU(),
        )

        channels = list(reversed(channels))
        self.ups = nn.ModuleList([convBlockGen(in_channels, channels[0], down=False, act="relu", use_dropout=True)])
        in_channels = channels[0]
        for channel_idx, channel in enumerate(channels[1:]):
            up = convBlockGen(in_channels*2, channel, down=False, act="relu", use_dropout=True if channel_idx<=1 else False)
            self.ups.append(up)
            in_channels = channel

        self.final_up = nn.Sequential(
            nn.ConvTranspose2d(in_channels*2, out_channels, kernel_size=4, stride=2, padding=1),
            nn.Tanh(),
        )

    def forward(self, x):
        x = self.initial_down(x)

        skip_connections = [x]
        for down in self.downs:
            x = down(x)
            skip_connections.append(x)

        x = self.bottleNeck(x)

        skip_connections = list(reversed(skip_connections))
        for i in range(7):
            x = self.ups[i](x)
            x = torch.cat((x, skip_connections[i]), dim=1)
        x = self.final_up(x)
        return x

In [7]:
x = torch.randn((8, 3, 256, 256))
gen = Generator()
gen(x).shape

torch.Size([8, 3, 256, 256])

In [8]:
class MapDataset:
    def __init__(self, img_paths, input_target_transform, input_transform, target_transform):
        self.img_paths = img_paths

    def __len__(self):
        return len(self.img_paths)

    def __getitem__(self, index):
        img_path = self.img_paths[index]
        img = np.array(Image.open(img_path))
        input_img = img[:, :600, :]
        target_img = img[:, 600:, :]

        aug = input_target_transform(image=input_img, image0=target_img)
        input_img = aug["image"]
        target_img = aug["image0"]

        input_img = input_transform(image=input_img)["image"]
        target_img = target_transform(image=target_img)["image"]

        return input_img, target_img

In [10]:
BATCH_SIZE = 16
IMG_SIZE = 256
NUM_WORKERS = 0

input_target_transform = A.Compose(
    [
        A.Resize(height=IMG_SIZE, width=IMG_SIZE),
        A.HorizontalFlip(0.5),
    ],
    additional_targets={"image0": "image"}
)

input_transform = A.Compose(
    [
        A.ColorJitter(p=0.2),
        A.Normalize(
            mean=[0.5, 0.5, 0.5], 
            std=[0.5, 0.5, 0.5], 
            max_pixel_value=255.0,
        ),
        ToTensorV2(),
    ]
)

target_transform = A.Compose(
    [
        A.Normalize(
            mean=[0.5, 0.5, 0.5], 
            std=[0.5, 0.5, 0.5], 
            max_pixel_value=255.0,
        ),
        ToTensorV2()    
    ]
)


train_dir = "./maps/train"
train_img_paths = [os.path.join(train_dir, img_name) for img_name in os.listdir(train_dir)]
train_dataset = MapDataset(train_img_paths, input_target_transform, input_transform, target_transform)
train_data_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS,
)

In [11]:
def save_checkpoint(model, optimizer, filename="model.pth.tar"):
    print("=>Saving Checkpoint...")
    checkpoint = {"model_state": model.state_dict(), "optimizer_state": optimizer.state_dict()}
    torch.save(checkpoint, filename)

def load_checkpoint(checkpoint_path, model, optimizer):
    print("=>Loading Checkpoint...")
    checkpoint = torch.load(checkpoint_path, map_location=torch.device(DEVICE))
    model.load_state_dict(checkpoint["model_state"])
    optimizer.load_state_dict(checkpoint["optimizer_state"])

In [12]:
def save_imgs(data_loader, gen, epoch, folder="./saved_imgs/"):
    gen.eval()
    for input_imgs, target_imgs in data_loader:
        input_imgs = input_imgs.to(DEVICE)
        target_imgs = target_imgs.to(DEVICE)

        with torch.no_grad():
            fake_imgs = gen(input_imgs)
        
        input_imgs = input_imgs * 0.5 + 0.5
        fake_imgs = fake_imgs * 0.5 + 0.5

        torchvision.utils.save_image(input_imgs, folder + f"input_{epoch}.png")
        torchvision.utils.save_image(fake_imgs, folder + f"fake_{epoch}.png")
        break

In [18]:
def train(train_data_loader, gen, disc, opt_gen, opt_disc, bce, l1_loss, gen_scaler, disc_scaler, epoch):
    gen.train()
    disc.train()

    train_progress_bar = tqdm(enumerate(train_data_loader), total=len(train_data_loader), leave=True)
    for idx, (input_imgs, target_imgs) in train_progress_bar:
        input_imgs = input_imgs.to(DEVICE)
        target_imgs = target_imgs.to(DEVICE)

        with torch.cuda.amp.autocast():
            fake_imgs = gen(input_imgs)
            outputs_real_imgs = disc(input_imgs, target_imgs)
            outputs_fake_imgs = disc(input_imgs, fake_imgs.detach())

            loss_disc_1 = bce(outputs_real_imgs, torch.ones_like(outputs_real_imgs))
            loss_disc_2 = bce(outputs_fake_imgs, torch.zeros_like(outputs_fake_imgs))
            loss_disc = (loss_disc_1 + loss_disc_2) / 2
        
        disc.zero_grad()
        disc_scaler.scale(loss_disc).backward()
        disc_scaler.step(opt_disc)
        disc_scaler.update()

        with torch.cuda.amp.autocast():
            outputs_fake_imgs = disc(input_imgs, fake_imgs)

            loss_gen_1 = bce(outputs_fake_imgs, torch.ones_like(outputs_fake_imgs)) 
            loss_gen_2 = l1_loss(fake_imgs, target_imgs) * L1_LAMBDA
            loss_gen = loss_gen_1 + loss_gen_2

        opt_gen.zero_grad()
        gen_scaler.scale(loss_gen).backward()
        gen_scaler.step(opt_gen)
        gen_scaler.update()

        train_progress_bar.set_description(f"Epoch [{epoch}/{EPOCHS-1}]")
        train_progress_bar.set_postfix(disc_loss=loss_disc.item(), gen_loss=loss_gen.item())
        
    train_progress_bar.close()

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
LEARNING_RATE = 2e-4

CHECKPOINT_GEN = "gen.pth.tar"
CHECKPOINT_DISC = "disc.pth.tar"
LOAD_GEN = True
LOAD_DISC = True

gen = Generator().to(DEVICE)
disc = Discriminator().to(DEVICE)

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

if LOAD_GEN:
    load_checkpoint(CHECKPOINT_GEN, gen, opt_gen)
if LOAD_DISC:
    load_checkpoint(CHECKPOINT_DISC, disc, opt_disc)

save_imgs(train_data_loader, gen, epoch=0)

=>Loading Checkpoint...
=>Loading Checkpoint...


In [19]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
L1_LAMBDA = 100

CHECKPOINT_GEN = "gen.pth.tar"
CHECKPOINT_DISC = "disc.pth.tar"
LOAD_GEN = False
LOAD_DISC = False

LEARNING_RATE = 2e-4
EPOCHS = 300

gen = Generator().to(DEVICE)
disc = Discriminator().to(DEVICE)

bce = nn.BCEWithLogitsLoss()
l1_loss = nn.L1Loss()

opt_gen = optim.Adam(gen.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))
opt_disc = optim.Adam(disc.parameters(), lr=LEARNING_RATE, betas=(0.5, 0.999))

gen_scaler = torch.cuda.amp.GradScaler()
disc_scaler = torch.cuda.amp.GradScaler()

if LOAD_GEN:
    load_checkpoint(CHECKPOINT_GEN, gen, opt_gen)
if LOAD_DISC:
    load_checkpoint(CHECKPOINT_DISC, disc, opt_disc)

for epoch in range(EPOCHS):
    train(train_data_loader, gen, disc, opt_gen, opt_disc, bce, l1_loss, gen_scaler, disc_scaler, epoch)

    save_checkpoint(gen, opt_gen, filename=CHECKPOINT_GEN)
    save_checkpoint(disc, opt_disc, filename=CHECKPOINT_DISC)
    save_imgs(train_data_loader, gen, epoch)

Epoch [0/299]: 100%|██████████| 69/69 [00:29<00:00,  2.35it/s, disc_loss=0.207, gen_loss=15.3]


=>Saving Checkpoint...
=>Saving Checkpoint...


Epoch [1/299]: 100%|██████████| 69/69 [00:29<00:00,  2.31it/s, disc_loss=0.628, gen_loss=13.3]


=>Saving Checkpoint...
=>Saving Checkpoint...


Epoch [2/299]: 100%|██████████| 69/69 [00:29<00:00,  2.36it/s, disc_loss=0.359, gen_loss=13.1]


...

=>Saving Checkpoint...
=>Saving Checkpoint...


Epoch [298/299]: 100%|██████████| 69/69 [00:30<00:00,  2.24it/s, disc_loss=0.187, gen_loss=10]


=>Saving Checkpoint...
=>Saving Checkpoint...


Epoch [299/299]: 100%|██████████| 69/69 [00:30<00:00,  2.24it/s, disc_loss=0.602, gen_loss=10.7]


=>Saving Checkpoint...
=>Saving Checkpoint...


In [36]:
gdrive_path = "/content/gdrive/MyDrive/googleColab/GANS/pix2pix"
shutil.make_archive(os.path.join(gdrive_path, "saved_imgs"), 'zip', "saved_imgs")
shutil.make_archive(os.path.join(gdrive_path, "gen"), 'zip', "gen")
shutil.make_archive(os.path.join(gdrive_path, "disc"), 'zip', "disc")

In [39]:
with ZipFile(os.path.join(gdrive_path, "gen.zip"), 'r') as z:
    z.extractall(path=gdrive_path)
    print("Done")

with ZipFile(os.path.join(gdrive_path, "disc.zip"), 'r') as z:
    z.extractall(path=gdrive_path)
    print("Done")

Done
